# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv data and create a dataframe.
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# Drop the NA that were inserted for cities without weather info
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.800000,-139.033333,78.37,76.0,20.0,16.69,PF,1.599978e+09
1,Puerto Ayora,-0.750000,-90.316667,66.00,93.0,95.0,4.00,EC,1.599978e+09
3,Altay,47.866667,88.116667,63.54,26.0,58.0,6.15,CN,1.599978e+09
4,Busselton,-33.652489,115.345497,69.01,41.0,14.0,5.46,AU,1.599978e+09
5,Bredasdorp,-34.532154,20.040308,59.00,67.0,61.0,12.75,ZA,1.599978e+09
...,...,...,...,...,...,...,...,...,...
995,Nantucket,41.283333,-70.100000,61.00,82.0,1.0,10.29,US,1.599978e+09
996,Sosua,19.750000,-70.516667,81.00,94.0,20.0,6.67,DO,1.599978e+09
997,Shahrud,36.418189,54.976282,74.26,23.0,0.0,7.83,IR,1.599978e+09
998,Synya,65.371849,58.038736,44.96,93.0,100.0,8.97,RU,1.599978e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

weather_criteria_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] == 0)]

print(f"The list of cities matching the criteria contains {len(weather_criteria_df)} cities.")
weather_criteria_df.reset_index(drop=True, inplace=True)
weather_criteria_df

The list of cities matching the criteria contains 32 cities.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Barentu,15.113889,37.592778,78.42,69.0,0.0,5.37,ER,1.599978e+09
1,Ladario,-19.016667,-57.583333,76.12,25.0,0.0,1.99,BR,1.599978e+09
2,Tahoua,14.890278,5.267778,78.80,83.0,0.0,4.70,NE,1.599978e+09
3,Rondonopolis,-16.466667,-54.633333,70.81,29.0,0.0,0.87,BR,1.599978e+09
4,Silvan,38.141944,41.008056,77.00,38.0,0.0,2.24,TR,1.599978e+09
5,Radovan,44.166667,23.616667,73.00,53.0,0.0,4.70,RO,1.599978e+09
6,Chibuto,-24.686667,33.530556,70.59,72.0,0.0,3.31,MZ,1.599978e+09
7,Komsomolskiy,69.166667,172.700000,75.20,27.0,0.0,4.70,RU,1.599978e+09
8,San Quintin,30.483333,-115.950000,70.56,77.0,0.0,9.91,MX,1.599978e+09
9,Azad Shahr,37.087009,55.173973,75.94,20.0,0.0,5.99,IR,1.599978e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

#### Setup the framework for the API call

* We are going to narrow the search for the first 10 cities.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_criteria_df.iloc[0:10,:].iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}, {row['Country']}...")

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Searching within Barentu, ER...


/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching within Ladario, BR...
Searching within Tahoua, NE...
Searching within Rondonopolis, BR...
Searching within Silvan, TR...
Searching within Radovan, RO...
Searching within Chibuto, MZ...
Searching within Komsomolskiy, RU...
Searching within San Quintin, MX...
Searching within Azad Shahr, IR...


In [8]:
# Drop the rows that contains NA leaving only the 10 hotels in the list with the desired columns
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']].dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Barentu,ER,15.113889,37.592778,Barentu
1,Ladario,BR,-19.016667,-57.583333,Corumbá
2,Tahoua,NE,14.890278,5.267778,Tahoua
3,Rondonopolis,BR,-16.466667,-54.633333,Rondonópolis
4,Silvan,TR,38.141944,41.008056,Silvan
5,Radovan,RO,44.166667,23.616667,Radovan
6,Chibuto,MZ,-24.686667,33.530556,Chibuto
7,Komsomolskiy,RU,69.166667,172.700000,Komsomolsky
8,San Quintin,MX,30.483333,-115.950000,Lázaro Cárdenas
9,Azad Shahr,IR,37.087009,55.173973,Azadshahr


In [9]:
# Export the hotels dataframe as csv
hotel_df.to_csv('data/hotels_list.csv',index=False)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hover_text=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))